# Future Air (LAEI 2025–2030) — Reproducible Notebook
This notebook reproduces the figures and outputs used in the Future Air section.

In [ ]:
!pip -q install pandas numpy matplotlib plotly openpyxl requests

In [ ]:
"""
Future Air (LAEI 2025–2030) — Reproducible Analysis Script
---------------------------------------------------------
Generates:
- Figure 1: headline scorecard (PNG)
- Figure 2a/2b: interactive choropleth maps (HTML)
- Figure 3: Top/Bottom 5 extremes table (PNG)
- Figure 4: Inner vs Outer mean % change (PNG)
- Figure 5: PM2.5 mechanism split (PNG)
- Clean output dataset (CSV)

Inputs:
- Extracted Data 2025&2030.xlsx  (your extracted LAEI tables)
- boroughs_london.geojson (London borough polygons)

If boroughs_london.geojson is missing, the script can download it
from Westminster Data Studio's open_data repository.

How to run:
    python future_air_analysis.py

Recommended environment:
    pip install pandas numpy matplotlib plotly openpyxl requests
"""

from __future__ import annotations

import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ---------------------------
# CONFIG (edit these paths)
# ---------------------------
INPUT_EXCEL = os.path.join("data", "Extracted Data 2025&2030.xlsx")

# Put the GeoJSON here (or let the script download it)
BOROUGHS_GEOJSON = os.path.join("data", "boroughs_london.geojson")

# Where outputs will be written
OUT_DIR = os.path.join("outputs", "future_air_figs")

# Download URL for GeoJSON (Westminster Data Studio)
# Source page: https://www.londondata.studio/data/boundary-files/
BOROUGHS_GEOJSON_URL = (
    "https://raw.githubusercontent.com/westminsterDataStudio/open_data/main/"
    "boundary_files/boroughs_london.geojson"
)

# ---------------------------
# Utilities
# ---------------------------
def ensure_geojson(path: str, url: str) -> None:
    """Download borough GeoJSON if it does not exist."""
    if os.path.exists(path):
        return
    os.makedirs(os.path.dirname(path), exist_ok=True)
    import requests

    print(f"[INFO] Downloading borough GeoJSON → {path}")
    r = requests.get(url, timeout=60)
    r.raise_for_status()
    with open(path, "wb") as f:
        f.write(r.content)

def parse_sheet(path: str, sheet_name: str) -> tuple[pd.DataFrame, list[str]]:
    """
    Read the Excel sheet and detect the header row where column B equals 'Source'.
    Returns:
        df: cleaned dataframe with a 'Source' column and borough columns
        borough_cols: list of borough column names
    """
    raw = pd.read_excel(path, sheet_name=sheet_name, header=None)

    header_row_idx = None
    for i in range(len(raw)):
        if raw.iloc[i, 1] == "Source":
            header_row_idx = i
            break
    if header_row_idx is None:
        raise ValueError(f"Could not detect header row in sheet: {sheet_name}")

    header = raw.iloc[header_row_idx].tolist()
    df = raw.iloc[header_row_idx + 1 :].copy()
    df.columns = header
    df = df[df["Source"].notna()].copy()

    exclude = {
        "Source",
        "Central London",
        "Inner London",
        "Outer London",
        "Non-GLA",
        "总计",
        "行标签",
    }
    borough_cols = [
        c for c in df.columns if isinstance(c, str) and c not in exclude
    ]

    # Coerce numeric columns
    for c in borough_cols + ["Central London", "Inner London", "Outer London", "Non-GLA", "总计"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    return df, borough_cols

def extract_borough_values(df: pd.DataFrame, borough_cols: list[str], source_name: str) -> pd.DataFrame:
    """Extract borough values for one LAEI 'Source' row."""
    sub = df[df["Source"] == source_name]
    if sub.empty:
        raise ValueError(f"Source '{source_name}' not found. Available sources: {df['Source'].unique()[:10]} ...")
    row = sub.iloc[0]
    vals = row[borough_cols].astype(float)
    return pd.DataFrame({"borough": borough_cols, "value": vals.values})

def merge_years(df25: pd.DataFrame, df30: pd.DataFrame, label: str) -> pd.DataFrame:
    """Merge 2025 vs 2030 borough values and compute changes."""
    m = df25.merge(df30, on="borough", suffixes=("_2025", "_2030"))
    m["abs_change"] = m["value_2030"] - m["value_2025"]
    m["pct_change"] = np.where(m["value_2025"] != 0, m["abs_change"] / m["value_2025"] * 100, np.nan)
    m["metric"] = label
    return m

def totals(df25: pd.DataFrame, df30: pd.DataFrame, source_name: str) -> tuple[float, float, float, float]:
    """London total ('总计') for one source in 2025 vs 2030."""
    r25 = df25[df25["Source"] == source_name].iloc[0]
    r30 = df30[df30["Source"] == source_name].iloc[0]
    total25 = float(r25["总计"])
    total30 = float(r30["总计"])
    abs_change = total30 - total25
    pct_change = abs_change / total25 * 100 if total25 else np.nan
    return total25, total30, abs_change, pct_change

def to_geo_name(b: str) -> str:
    """Match borough naming between LAEI extract and GeoJSON."""
    if b == "Kingston":
        return "Kingston upon Thames"
    if b == "Richmond":
        return "Richmond upon Thames"
    return b

# London Plan (2021) Annex 2: Inner/Outer boroughs
INNER_BOROUGHS = {
    "City of London",
    "Camden",
    "Greenwich",
    "Hackney",
    "Hammersmith and Fulham",
    "Islington",
    "Kensington and Chelsea",
    "Lambeth",
    "Lewisham",
    "Newham",
    "Southwark",
    "Tower Hamlets",
    "Wandsworth",
    "Westminster",
}

def add_inner_outer(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    out["zone"] = np.where(out["borough"].isin(INNER_BOROUGHS), "Inner London", "Outer London")
    return out

# ---------------------------
# Main analysis
# ---------------------------
def main() -> None:
    os.makedirs(OUT_DIR, exist_ok=True)
    ensure_geojson(BOROUGHS_GEOJSON, BOROUGHS_GEOJSON_URL)

    # Load LAEI extracts
    df_nox25, borough_cols = parse_sheet(INPUT_EXCEL, "Nox Emission 2025")
    df_nox30, _ = parse_sheet(INPUT_EXCEL, "Nox Emission 2030")
    df_pm25_25, _ = parse_sheet(INPUT_EXCEL, "PM2.5 2025")
    df_pm25_30, _ = parse_sheet(INPUT_EXCEL, "PM2.5 2030")

    # Extract sources
    rt_nox25 = extract_borough_values(df_nox25, borough_cols, "Road Transport")
    rt_nox30 = extract_borough_values(df_nox30, borough_cols, "Road Transport")
    rt_pm25_25 = extract_borough_values(df_pm25_25, borough_cols, "Road Transport")
    rt_pm25_30 = extract_borough_values(df_pm25_30, borough_cols, "Road Transport")
    res_pm25_25 = extract_borough_values(df_pm25_25, borough_cols, "Resuspension")
    res_pm25_30 = extract_borough_values(df_pm25_30, borough_cols, "Resuspension")

    # Merge years + compute changes
    m_rt_nox = merge_years(rt_nox25, rt_nox30, "Road Transport NOx")
    m_rt_pm25 = merge_years(rt_pm25_25, rt_pm25_30, "Road Transport PM2.5")
    m_res_pm25 = merge_years(res_pm25_25, res_pm25_30, "Resuspension PM2.5")

    # London totals for scorecard
    tot_rt_nox = totals(df_nox25, df_nox30, "Road Transport")
    tot_rt_pm25 = totals(df_pm25_25, df_pm25_30, "Road Transport")
    tot_res_pm25 = totals(df_pm25_25, df_pm25_30, "Resuspension")

    # ---------------------------
    # Figure 1 — Scorecard table
    # ---------------------------
    scorecard = pd.DataFrame(
        [
            ["Road Transport NOx", *tot_rt_nox],
            ["Road Transport PM2.5", *tot_rt_pm25],
            ["Resuspension PM2.5", *tot_res_pm25],
        ],
        columns=["Metric", "Total 2025", "Total 2030", "Absolute change", "% change"],
    )
    scorecard_fmt = scorecard.copy()
    scorecard_fmt["Total 2025"] = scorecard_fmt["Total 2025"].map(lambda x: f"{x:,.1f}")
    scorecard_fmt["Total 2030"] = scorecard_fmt["Total 2030"].map(lambda x: f"{x:,.1f}")
    scorecard_fmt["Absolute change"] = scorecard_fmt["Absolute change"].map(lambda x: f"{x:,.1f}")
    scorecard_fmt["% change"] = scorecard_fmt["% change"].map(lambda x: f"{x:.1f}%")

    fig = plt.figure(figsize=(10, 2.2))
    ax = fig.add_subplot(111)
    ax.axis("off")
    tbl = ax.table(
        cellText=scorecard_fmt.values,
        colLabels=scorecard_fmt.columns,
        cellLoc="center",
        loc="center",
    )
    tbl.auto_set_font_size(False)
    tbl.set_fontsize(10)
    tbl.scale(1, 1.5)
    plt.title("Figure 1 — Headline changes (London total), 2025 → 2030", pad=10)
    fig1_path = os.path.join(OUT_DIR, "fig1_scorecard.png")
    plt.savefig(fig1_path, dpi=220, bbox_inches="tight")
    plt.close(fig)

    # ---------------------------
    # Figure 3 — Top/Bottom 5 extremes (table)
    # ---------------------------
    def top_bottom(df: pd.DataFrame, n: int = 5):
        s = df[["borough", "pct_change"]].sort_values("pct_change")
        top = s.head(n).reset_index(drop=True)
        bottom = s.tail(n).reset_index(drop=True)
        return top, bottom

    top_nox, bottom_nox = top_bottom(m_rt_nox, 5)
    top_pm, bottom_pm = top_bottom(m_rt_pm25, 5)

    tb = pd.DataFrame(
        {
            "NOx — Top 5 (largest reductions)": top_nox["borough"],
            "NOx %Δ": top_nox["pct_change"].map(lambda x: f"{x:.1f}%"),
            "NOx — Bottom 5 (smallest reductions)": bottom_nox["borough"],
            "NOx %Δ ": bottom_nox["pct_change"].map(lambda x: f"{x:.1f}%"),
            "PM2.5 — Top 5 (largest reductions)": top_pm["borough"],
            "PM2.5 %Δ": top_pm["pct_change"].map(lambda x: f"{x:.1f}%"),
            "PM2.5 — Bottom 5 (smallest reductions)": bottom_pm["borough"],
            "PM2.5 %Δ ": bottom_pm["pct_change"].map(lambda x: f"{x:.1f}%"),
        }
    )

    fig = plt.figure(figsize=(14, 2.8))
    ax = fig.add_subplot(111)
    ax.axis("off")
    tbl = ax.table(
        cellText=tb.values, colLabels=tb.columns, cellLoc="center", loc="center"
    )
    tbl.auto_set_font_size(False)
    tbl.set_fontsize(9)
    tbl.scale(1, 1.5)
    plt.title(
        "Figure 3 — Borough extremes in % change (Road Transport), 2025 → 2030",
        pad=10,
    )
    fig3_path = os.path.join(OUT_DIR, "fig3_top_bottom5_table.png")
    plt.savefig(fig3_path, dpi=220, bbox_inches="tight")
    plt.close(fig)

    # ---------------------------
    # Figure 4 — Inner vs Outer (mean % change)
    # ---------------------------
    m_rt_nox_io = add_inner_outer(m_rt_nox)
    m_rt_pm_io = add_inner_outer(m_rt_pm25)

    means = pd.DataFrame(
        {
            "Road Transport NOx": m_rt_nox_io.groupby("zone")["pct_change"].mean(),
            "Road Transport PM2.5": m_rt_pm_io.groupby("zone")["pct_change"].mean(),
        }
    ).loc[["Inner London", "Outer London"]]

    fig = plt.figure(figsize=(7.5, 4.8))
    ax = fig.add_subplot(111)
    means.plot(kind="bar", ax=ax)
    ax.set_ylabel("Mean % change (2030 vs 2025)")
    ax.set_xlabel("")
    ax.set_title("Figure 4 — Inner vs Outer London (mean % change)")
    ax.axhline(0, linewidth=1)
    plt.xticks(rotation=0)
    plt.tight_layout()
    fig4_path = os.path.join(OUT_DIR, "fig4_inner_outer_mean_pctchange.png")
    plt.savefig(fig4_path, dpi=220, bbox_inches="tight")
    plt.close(fig)

    # ---------------------------
    # Figure 5 — PM2.5 mechanism split (Road Transport vs Resuspension totals)
    # ---------------------------
    pm_mech = pd.DataFrame(
        {"2025": [tot_rt_pm25[0], tot_res_pm25[0]], "2030": [tot_rt_pm25[1], tot_res_pm25[1]]},
        index=["PM2.5 — Road Transport", "PM2.5 — Resuspension"],
    )

    fig = plt.figure(figsize=(8.5, 4.8))
    ax = fig.add_subplot(111)
    pm_mech.plot(kind="bar", ax=ax)
    ax.set_ylabel("Total emissions (tonnes/yr)")
    ax.set_xlabel("")
    ax.set_title("Figure 5 — PM2.5 pathways: tailpipe vs resuspension (London total)")
    plt.xticks(rotation=0)
    plt.tight_layout()
    fig5_path = os.path.join(OUT_DIR, "fig5_pm25_mechanism_split.png")
    plt.savefig(fig5_path, dpi=220, bbox_inches="tight")
    plt.close(fig)

    # ---------------------------
    # Figure 2 — Interactive choropleths (HTML)
    # ---------------------------
    import plotly.express as px

    with open(BOROUGHS_GEOJSON, "r", encoding="utf-8") as f:
        gj = json.load(f)

    map_nox2030 = m_rt_nox[["borough", "value_2030"]].copy()
    map_nox2030["borough_geo"] = map_nox2030["borough"].map(to_geo_name)

    map_pm2030 = m_rt_pm25[["borough", "value_2030"]].copy()
    map_pm2030["borough_geo"] = map_pm2030["borough"].map(to_geo_name)

    fig_nox = px.choropleth(
        map_nox2030,
        geojson=gj,
        locations="borough_geo",
        featureidkey="properties.NAME",
        color="value_2030",
        projection="mercator",
        title="Figure 2a — Road Transport NOx (tonnes/yr), 2030",
    )
    fig_nox.update_geos(fitbounds="locations", visible=False)

    fig_pm = px.choropleth(
        map_pm2030,
        geojson=gj,
        locations="borough_geo",
        featureidkey="properties.NAME",
        color="value_2030",
        projection="mercator",
        title="Figure 2b — Road Transport PM2.5 (tonnes/yr), 2030",
    )
    fig_pm.update_geos(fitbounds="locations", visible=False)

    fig2a_path = os.path.join(OUT_DIR, "fig2a_map_nox_2030.html")
    fig2b_path = os.path.join(OUT_DIR, "fig2b_map_pm25_2030.html")
    fig_nox.write_html(fig2a_path, include_plotlyjs="cdn")
    fig_pm.write_html(fig2b_path, include_plotlyjs="cdn")

    # ---------------------------
    # Export analysis-ready dataset (CSV) for GitHub / Data page
    # ---------------------------
    future_air_borough = (
        m_rt_nox[["borough", "value_2025", "value_2030", "abs_change", "pct_change"]]
        .rename(
            columns={
                "value_2025": "nox_rt_2025",
                "value_2030": "nox_rt_2030",
                "abs_change": "nox_rt_abs",
                "pct_change": "nox_rt_pct",
            }
        )
        .merge(
            m_rt_pm25[["borough", "value_2025", "value_2030", "abs_change", "pct_change"]].rename(
                columns={
                    "value_2025": "pm25_rt_2025",
                    "value_2030": "pm25_rt_2030",
                    "abs_change": "pm25_rt_abs",
                    "pct_change": "pm25_rt_pct",
                }
            ),
            on="borough",
        )
        .merge(
            m_res_pm25[["borough", "value_2025", "value_2030", "abs_change", "pct_change"]].rename(
                columns={
                    "value_2025": "pm25_res_2025",
                    "value_2030": "pm25_res_2030",
                    "abs_change": "pm25_res_abs",
                    "pct_change": "pm25_res_pct",
                }
            ),
            on="borough",
        )
    )
    future_air_borough["zone"] = np.where(
        future_air_borough["borough"].isin(INNER_BOROUGHS), "Inner London", "Outer London"
    )

    csv_path = os.path.join(OUT_DIR, "future_air_borough_2025_2030_rt_resusp.csv")
    future_air_borough.to_csv(csv_path, index=False)

    print("[DONE] Outputs written to:", OUT_DIR)
    print(" -", os.path.basename(fig1_path))
    print(" -", os.path.basename(fig2a_path))
    print(" -", os.path.basename(fig2b_path))
    print(" -", os.path.basename(fig3_path))
    print(" -", os.path.basename(fig4_path))
    print(" -", os.path.basename(fig5_path))
    print(" -", os.path.basename(csv_path))

main()
